In [1]:
!pwd
%ls
%cd /
%ls

/mnt/workspace
glm3ft.ipynb  ZhipuAI-chatglm3-6b.ipynb
/
'=0.21.0'                     dev/     libx32@                    proc/   tmp/
'=0.6.0'                      etc/     media/                     root/   usr/
'=2.25.0'                     home/    mnt/                       run/    var/
 bin@                         lib@     modelscope/                sbin@
 boot/                        lib32@   NGC-DL-CONTAINER-LICENSE   srv/
 cuda-keyring_1.0-1_all.deb   lib64@   opt/                       sys/


In [1]:
!nvidia-smi

Fri May  3 15:28:47 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 12.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10          On   | 00000000:00:07.0 Off |                    0 |
|  0%   33C    P8    22W / 150W |      0MiB / 22731MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
pip install 'accelerate>=0.27.2'

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.5 MB/s eta 0:00:00a 0:00:01
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.26.1
    Uninstalling accelerate-0.26.1:
      Successfully uninstalled accelerate-0.26.1
Note: you may need to restart the kernel to use updated packages.


# 下载模型


In [1]:
from modelscope import snapshot_download
model_dir = snapshot_download("ZhipuAI/chatglm3-6b", cache_dir='chatglm3-6b', revision = "master")


2024-05-07 17:59:03,073 - modelscope - INFO - PyTorch version 2.1.2+cu121 Found.
2024-05-07 17:59:03,077 - modelscope - INFO - TensorFlow version 2.14.0 Found.
2024-05-07 17:59:03,077 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2024-05-07 17:59:03,078 - modelscope - INFO - No valid ast index found from /mnt/workspace/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2024-05-07 17:59:03,153 - modelscope - INFO - Loading done! Current index file version is 1.14.0, with md5 11536c73f9a69de5892dd60259abcf6a and a total number of 976 components indexed
/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-07 17:59:04,955 - modelscope - INFO - Development mode use revision: master
Downloading: 100%|██████████| 1.29k/1.29k [00:00<00:00

In [ ]:
!git config --global http.postBuffer 524288000 && git clone https://github.com/hiyouga/LLaMA-Factory.git
!git config --global http.postBuffer 524288000 && git clone https://github.com/kangtsang/ft_data.git

正克隆到 'LLaMA-Factory'...
remote: Enumerating objects: 10953, done.
remote: Counting objects: 100% (1770/1770), done.
remote: Compressing objects: 100% (428/428), done.
remote: Total 10953 (delta 1436), reused 1573 (delta 1332), pack-reused 9183
接收对象中: 100% (10953/10953), 214.56 MiB | 11.33 MiB/s, 完成.
处理 delta 中: 100% (8035/8035), 完成.
正在更新文件: 100% (216/216), 完成.
正克隆到 'ft_data'...
remote: Enumerating objects: 449, done.
remote: Counting objects: 100% (218/218), done.
remote: Compressing objects: 100% (166/166), done.


In [13]:
!cd LLaMA-Factory && pwd && git checkout -b 'master'

/mnt/workspace/LLaMA-Factory
切换到一个新分支 'master'


In [ ]:
import tensorrt as tr
import tensorflow as tf
import torch

print(f"tf cuda available：{tf.config.list_physical_devices('GPU')}")

# 输出带CPU，表示torch是CPU版本的，否则会是+cuxxx
print(f'torch的版本是：{torch.__version__}')
print(f'torch是否能使用cuda：{torch.cuda.is_available()}')

In [ ]:
!cd LLaMA-Factory && pwd && pip install -e .[metrics,vllm]

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python LLaMA-Factory/src/train.py \
    --stage sft \
    --do_train True \
    --model_name_or_path /mnt/workspace/chatglm3-6b/ZhipuAI/chatglm3-6b \
    --quantization_bit 8 \
    --finetuning_type lora \
    --template chatglm3 \
    --dataset_dir ft_data \
    --dataset ad_gen \
    --cutoff_len 1024 \
    --learning_rate 5e-05 \
    --num_train_epochs 3.0 \
    --max_samples 1000 \
    --per_device_train_batch_size 35 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 100 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --report_to none \
    --output_dir saves/glm3_train_2024-05-03 \
    --fp16 True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.1 \
    --lora_target query_key_value \
    --plot_loss True 

In [2]:
!pwd
# /mnt/workspace
!cd LLaMA-Factory   
!pwd

/mnt/workspace
/mnt/workspace


In [ ]:
# 模型对话 /mnt/workspace/chatglm3-6b
from llmtuner.chat import ChatModel
from llmtuner.extras.misc import torch_gc
chat_model = ChatModel(dict(
  model_name_or_path="/mnt/workspace/chatglm3-6b/ZhipuAI/chatglm3-6b",     # 使用 4 比特量化版 Llama-3-8b-Instruct 模型
  adapter_name_or_path="ft_data/train/glm3_train_classify_2024-05-06",   # 加载之前保存的 LoRA 适配器
  finetuning_type="lora",                  # 和训练保持一致
  template="chatglm3",                     # 和训练保持一致
  # quantization_bit=8,           # 加载 4 比特量化模型
))
messages = []
while True:
  query = input("\nUser: ")
  if query.strip() == "exit":
    break

  if query.strip() == "clear":
    messages = []
    torch_gc()
    print("History has been removed.")
    continue

  messages.append({"role": "user", "content": query})     # 把提示词添加到消息中
  print("Assistant: ", end="", flush=True)
  response = ""
  for new_text in chat_model.stream_chat(messages):      # 流式输出
    print(new_text, end="", flush=True)
    response += new_text
  print()
  messages.append({"role": "assistant", "content": response}) # 把回答添加到消息中

torch_gc()

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python LLaMA-Factory/src/train.py \
    --stage sft \
    --model_name_or_path /mnt/workspace/chatglm3-6b/ZhipuAI/chatglm3-6b \
    --adapter_name_or_path /mnt/workspace/ft_data/train/glm3_train_classify_2024-05-06 \
    --finetuning_type lora \
    --template chatglm3 \
    --flash_attn auto \
    --dataset_dir ft_data \
    --dataset classify_dev \
    --cutoff_len 1024 \
    --max_samples 100000 \
    --per_device_eval_batch_size 8 \
    --predict_with_generate True \
    --max_new_tokens 128 \
    --top_p 0.7 \
    --temperature 0.1 \
    --output_dir saves/glm3_keyword_dev_2024-05-06 \
    --do_predict True 

2024-05-07 18:08:49.800286: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-07 18:08:49.847344: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-07 18:08:49.847378: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-07 18:08:49.847418: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-07 18:08:49.855844: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-07 18:08:49.856131: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python LLaMA-Factory/src/train.py \
    --stage sft \
    --model_name_or_path /mnt/workspace/chatglm3-6b/ZhipuAI/chatglm3-6b \
    --adapter_name_or_path /mnt/workspace/ft_data\train\glm3_train_keyword_2024-05-06 \
    --finetuning_type lora \
    --template chatglm3 \
    --flash_attn auto \
    --dataset_dir ft_data \
    --dataset classify_dev \
    --cutoff_len 1024 \
    --max_samples 100000 \
    --per_device_eval_batch_size 8 \
    --predict_with_generate True \
    --max_new_tokens 128 \
    --top_p 0.7 \
    --temperature 0.1 \
    --output_dir saves/glm3_keyword_dev_2024-05-06 \
    --do_predict True 

In [ ]:
!zip -q -r "saves_2024-05-07.zip" /mnt/workspace/saves